# Huffman Coding

In [3]:
class HeapNode
{
    HeapNode* left = nullptr;
    HeapNode* right = nullptr;
    double freq;
    char ch;

public: 
    HeapNode(const double& freq, const char& ch, HeapNode* left=nullptr, HeapNode* right=nullptr) 
        : freq(freq), ch(ch), left(left), right(right) {}
    
    bool operator<(const HeapNode& other){
        return this->freq < other.freq;
    }
};

In [2]:
#include <string>
#include <queue>
#include <vector>
#include <functional>
#include <unordered_map>

std::string message(R"(In computer science and information theory, a Huffman code is a particular type of optimal prefix code that is commonly used for lossless data compression. The process of finding or using such a code proceeds by means of Huffman coding, an algorithm developed by David A. Huffman while he was a Sc.D. student at MIT, and published in the 1952 paper "A Method for the Construction of Minimum-Redundancy Codes".)");

std::unordered_map<char, int> freq;
for (const char& c : message)
    freq[c]++;

std::vector<HeapNode> nodes;
for (auto [letter, frequency] : freq)
    nodes.push_back(HeapNode(frequency, letter));

std::priority_queue<HeapNode> q(std::less<HeapNode>(), nodes);

q

In file included from input_line_5:1:
In file included from /srv/conda/envs/notebook/include/xeus/xinterpreter.hpp:13:
In file included from /usr/lib/gcc/x86_64-linux-gnu/7.5.0/../../../../include/c++/7.5.0/functional:49:
/usr/lib/gcc/x86_64-linux-gnu/7.5.0/../../../../include/c++/7.5.0/bits/stl_function.h:386:20: error: invalid operands to binary expression ('const __cling_N52::HeapNode' and 'const __cling_N52::HeapNode')
      { return __x < __y; }
               ~~~ ^ ~~~
/usr/lib/gcc/x86_64-linux-gnu/7.5.0/../../../../include/c++/7.5.0/bits/predefined_ops.h:143:23: note: in instantiation of member function 'std::less<__cling_N52::HeapNode>::operator()' requested here
        { return bool(_M_comp(*__it1, *__it2)); }
                      ^
/usr/lib/gcc/x86_64-linux-gnu/7.5.0/../../../../include/c++/7.5.0/bits/stl_heap.h:222:8: note: in instantiation of function template specialization '__gnu_cxx::__ops::_Iter_comp_iter<std::less<__cling_N52::HeapNode> >::operator()<__gnu_cxx::__nor

Interpreter Error: 